In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
fetal_health = pd.read_csv('/kaggle/input/fetal-health-classification/fetal_health.csv')

In [ ]:
fetal_health.head()

In [ ]:
fetal_health['fetal_health'].value_counts()

In [ ]:
import seaborn as sns
from sklearn import model_selection
import matplotlib.pyplot as plt

import tensorflow as tf

In [ ]:
print(tf.__version__)

In [ ]:
fetal_health.describe().T

In [ ]:
fetal_health.info()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(fetal_health.corr())

In [ ]:
X = fetal_health.drop(['fetal_health'], axis=1).values
y = fetal_health['fetal_health'].values

In [ ]:
seed = 42

X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.3, random_state=seed)
X_val, X_test, y_val, y_test = model_selection.train_test_split(X_val, y_val, test_size=0.5, random_state=seed)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
rows, feat = X_train.shape

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(feat,),),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(2048, activation='relu', kernel_initializer='normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation='relu', kernel_initializer='normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu', kernel_initializer='normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [ ]:
print(model.summary)

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

adam = tf.keras.optimizers.Adam(lr=0.0001)

model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train,
                validation_data = (X_val, y_val),
                epochs=600,
                batch_size=32,
                callbacks=[earlystop],
                verbose=1)

In [ ]:
plt.plot(hist.history['loss'], label='training loss')
plt.plot(hist.history['val_loss'], label='validation loss')
plt.legend()

In [ ]:
plt.plot(hist.history['accuracy'], label='training accuracy')
plt.plot(hist.history['val_accuracy'], label='validation accuracy')
plt.legend()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score

categorical_pred = np.argmax(model.predict(X_test), axis=1)

print('Results for Categorical Mode')
print(accuracy_score(y_test, categorical_pred))
print(classification_report(y_test, categorical_pred))

In [ ]:
print(confusion_matrix(y_test, categorical_pred))

In [ ]:
predictions = model.predict(X_test)

parr = []

for pred in predictions:
    parr.append(np.argmax(pred))


In [ ]:
parr = np.array(parr).reshape(-1)
print("F1 score {}".format(f1_score(y_test, parr, average='micro')))